In [ ]:
```xml
<VSCode.Cell language="markdown">
# SPR 2026 - TF-IDF + LightGBM (TRATADO)

**Versão tratada do 4º melhor modelo (0.70273)**

**Tratamento aplicado:**
- ✅ Normalização de texto (lowercase, acentos)
- ✅ Remoção de stop words (preservando termos médicos)
- ✅ Preservação de termos BI-RADS
- ✅ Limpeza de caracteres especiais

**Optimizações:**
- ✅ TruncatedSVD: 15k features → 500 features densas
- ✅ GPU acceleration

**Hipótese:** Laudos muito parecidos → tratamento cuidadoso pode ajudar a capturar diferenças sutis.

---
**CONFIGURAÇÃO KAGGLE:**
1. Settings → Internet → **OFF**
2. Settings → Accelerator → **GPU T4 x2** (recomendado)
3. **IMPORTANTE:** Execute "Run All" após commit
---
</VSCode.Cell>
<VSCode.Cell language="python">
# =============================================================================
# SPR 2026 - TFIDF + LIGHTGBM (VERSÃO TRATADA)
# =============================================================================
# Teste de hipótese: tratamento de texto melhora performance?
# Baseline: 0.70273 (sem tratamento)
# =============================================================================

import numpy as np
import pandas as pd
import re
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import torch
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - TF-IDF + LightGBM (TRATADO)")
print("="*60)

# ==== CONFIGURAÇÕES ====
SEED = 42
SVD_COMPONENTS = 500
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
USE_GPU = torch.cuda.is_available()
np.random.seed(SEED)

print(f"✓ GPU disponível: {USE_GPU}")

# =============================================================================
# FUNÇÕES DE TRATAMENTO DE TEXTO
# =============================================================================

# Stop words em português (excluindo termos médicos importantes)
STOP_WORDS_PT = {
    'a', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo',
    'as', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele',
    'deles', 'depois', 'do', 'dos', 'e', 'ela', 'elas', 'ele', 'eles', 'em',
    'entre', 'era', 'eram', 'essa', 'essas', 'esse', 'esses', 'esta', 'estas',
    'este', 'estes', 'eu', 'foi', 'fomos', 'for', 'foram', 'há', 'isso',
    'isto', 'já', 'lhe', 'lhes', 'lo', 'mas', 'me', 'mesmo', 'meu', 'meus',
    'minha', 'minhas', 'muito', 'na', 'nas', 'nem', 'no', 'nos', 'nossa',
    'nossas', 'nosso', 'nossos', 'num', 'numa', 'nuns', 'o', 'os', 'ou',
    'para', 'pela', 'pelas', 'pelo', 'pelos', 'por', 'qual', 'quando',
    'que', 'quem', 'são', 'se', 'seja', 'sejam', 'sem', 'seu', 'seus',
    'só', 'sua', 'suas', 'também', 'te', 'tem', 'tendo', 'tenho', 'ter',
    'teu', 'teus', 'ti', 'tive', 'tivemos', 'tiveram', 'tu', 'tua', 'tuas',
    'um', 'uma', 'umas', 'uns', 'você', 'vocês', 'vos'
}

# Termos BI-RADS importantes (NÃO remover)
BIRADS_TERMS = {
    'birads', 'bi-rads', 'categoria', 'calcificacao', 'calcificacoes',
    'nodulo', 'nodulos', 'massa', 'massas', 'assimetria', 'assimetrias',
    'distorcao', 'densidade', 'benigno', 'benigna', 'maligno', 'maligna',
    'suspeito', 'suspeita', 'provavelmente', 'tipicamente', 'achado',
    'achados', 'mama', 'mamas', 'mamografia', 'ultrassom', 'ecografia',
    'axila', 'axilar', 'linfonodo', 'linfonodos', 'pele', 'mamilo',
    'areola', 'parenquima', 'fibroglandular', 'adiposo', 'heterogeneo',
    'homogeneo', 'denso', 'densa', 'espiculado', 'circunscrito', 'irregular',
    'oval', 'redondo', 'lobulado', 'microlobulado', 'obscurecido',
    'parcialmente', 'totalmente', 'grosseiras', 'finas', 'pleomorficas',
    'amorfas', 'puntiformes', 'lineares', 'ramificadas', 'segmentar',
    'regional', 'difuso', 'agrupadas', 'clusters', 'estavel', 'novo',
    'aumentou', 'diminuiu', 'inalterado', 'recomenda', 'biopsia',
    'controle', 'seguimento', 'rotina', 'complementar', 'negativo',
    'positivo', 'inconclusivo'
}

def remove_accents(text):
    """Remove acentos do texto."""
    nfkd = unicodedata.normalize('NFKD', text)
    return ''.join(c for c in nfkd if not unicodedata.combining(c))

def preprocess_text(text):
    """
    Pré-processa texto de laudo de mamografia.
    - Lowercase
    - Remove acentos
    - Remove caracteres especiais (mantém hífen para BI-RADS)
    - Remove stop words (preservando termos médicos)
    """
    if pd.isna(text):
        return ""
    
    # Lowercase
    text = text.lower()
    
    # Remove acentos
    text = remove_accents(text)
    
    # Normaliza BI-RADS
    text = re.sub(r'bi[\s-]*rads?', 'birads', text)
    
    # Remove caracteres especiais (mantém letras, números e espaços)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    
    # Remove números isolados (mas mantém medidas como contexto)
    text = re.sub(r'\b\d+\b', '', text)
    
    # Remove espaços extras
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove stop words (preservando termos BI-RADS)
    words = text.split()
    words = [w for w in words if w not in STOP_WORDS_PT or w in BIRADS_TERMS]
    
    return ' '.join(words)

# ==== CARREGAR DADOS ====
print("\n[1/6] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f"    Train: {train.shape} | Test: {test.shape}")

# ==== TRATAMENTO DE TEXTO ====
print("\n[2/6] Aplicando tratamento de texto...")
train['report_treated'] = train['report'].apply(preprocess_text)
test['report_treated'] = test['report'].apply(preprocess_text)

# Mostrar exemplo
print("\n    Exemplo de tratamento:")
print(f"    Original: {train['report'].iloc[0][:100]}...")
print(f"    Tratado:  {train['report_treated'].iloc[0][:100]}...")

# ==== TF-IDF ====
print("\n[3/6] Aplicando TF-IDF...")
tfidf = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)
X_train_tfidf = tfidf.fit_transform(train['report_treated'])
X_test_tfidf = tfidf.transform(test['report_treated'])
y_train = train['target'].values
print(f"    TF-IDF esparso: {X_train_tfidf.shape}")

# ==== SVD ====
print(f"\n[4/6] Aplicando SVD: {X_train_tfidf.shape[1]} → {SVD_COMPONENTS} features...")
svd = TruncatedSVD(n_components=SVD_COMPONENTS, random_state=SEED)
X_train_svd = svd.fit_transform(X_train_tfidf)
X_test_svd = svd.transform(X_test_tfidf)
print(f"    Variância explicada: {svd.explained_variance_ratio_.sum():.2%}")
print(f"    ✅ Shape denso: {X_train_svd.shape}")

# Normalizar
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_svd)
X_test = scaler.transform(X_test_svd)

# ==== SPLIT PARA EARLY STOPPING ====
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=SEED, stratify=y_train
)

# ==== LIGHTGBM ====
print(f"\n[5/6] Treinando LightGBM ({'GPU' if USE_GPU else 'CPU'})...")
model = lgb.LGBMClassifier(
    n_estimators=300,
    max_depth=8,
    learning_rate=0.1,
    num_leaves=31,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight='balanced',
    device='gpu' if USE_GPU else 'cpu',
    random_state=SEED,
    n_jobs=-1,
    verbose=-1
)

model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    callbacks=[
        lgb.early_stopping(stopping_rounds=30, verbose=True),
        lgb.log_evaluation(period=50)
    ]
)
print(f"    ✓ Melhor iteração: {model.best_iteration_}")
print("    ✓ Modelo treinado!")

# ==== SUBMISSÃO ====
print("\n[6/6] Gerando submissão...")
predictions = model.predict(X_test)

submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})
submission.to_csv('submission.csv', index=False)

print("="*60)
print("✅ CONCLUÍDO - submission.csv criado!")
print("="*60)
print("\nDistribuição das predições:")
print(submission['target'].value_counts().sort_index())
</VSCode.Cell>
```